In [1]:
using TensorFlow
TensorFlow.tf_version()

v"1.10.0"

# Tensors

TensorFlow, as the name indicates, is a framework to define and run computations involving tensors. A **tensor** is a generalization of vectors and matrices to potentially higher dimensions. Internally, TensorFlow represents tensors as n-dimensional arrays of base datatypes.

When writing a TensorFlow program, the main object you manipulate and pass around is the `Tensor`. A `Tensor` object represents a partially defined computation that will eventually produce a value. TensorFlow programs work by first building a graph of `Tensor` objects, detailing how each tensor is computed based on the other available tensors and then by running parts of this graph to achieve the desired results.

A `Tensor` has the following properties:

* a data type (`Float32`, `Int32`, or `String`, for example)

* a shape

Each element in the Tensor has the same data type, and the data type is always known. The shape (that is, the number of dimensions it has and the size of each dimension) might be only partially known. Most operations produce tensors of fully-known shapes if the shapes of their inputs are also fully known, but in some cases it's only possible to find the shape of a tensor at graph execution time.

Some types of tensors are special, and these will be covered in other units of the TensorFlow guide. The main ones are:

* `Variable`
* `constant`
* `placeholder`
* `SparseTensor` (not yet supported by TensorFlow.jl)

With the exception of `Variable`, the value of a tensor is immutable, which means that in the context of a single execution tensors only have a single value. However, evaluating the same tensor twice can return different values; for example that tensor can be the result of reading data from disk, or generating a random number.

## Rank

The **rank** of a `Tensor` object is its number of dimensions. Synonyms for rank include **order** or **degree** or **n-dimension**. Note that rank in TensorFlow is not the same as matrix rank in mathematics. As the following table shows, each rank in TensorFlow corresponds to a different mathematical entity:

| Rank | Math entity |
| 0 | Scalar (magnitude only) |
| 1 | Vector (magnitude and direction) |
| 2 | Matrix (table of numbers) |
| 3 | 3-Tensor (cube of numbers) |
| n | n-Tensor (you get the idea) |

### Rank 0

The following snippet demonstrates creating a few rank 0 variables:

In [32]:
# mammal = Variable("Elephant") # currently errors
ignition = Variable(451)
floating = Variable(3.14159265359)
its_complicated = Variable(12.3 - 4.85im);

### Rank 1

To create a rank 1 `Tensor` object, you can pass a vector of items as the initial value. For example:

In [36]:
mystr = Variable(["Hello"])
cool_numbers = Variable([3.14159, 2.71828])
first_primes = Variable([2, 3, 5, 7, 11])
its_very_complicated = Variable([12.3 - 4.85im, 7.5 - 6.23im]);

### Higher ranks

A rank 2 `Tensor` object consists of at least one row and at least one column:

In [52]:
mymat = Variable([7 11])
myxor = Variable([false true; true false])
linear_squares = Variable([4 9 16 25])
squarish_squares = Variable([4 9; 16 25]);

Higher-rank Tensors, similarly, consist of an n-dimensional array. For example, during image processing, many tensors of rank 4 are used, with dimensions corresponding to example-in-batch, image width, image height, and color channel.

In [61]:
my_image = Variable(zeros(10, 299, 299, 3)); # batch x height x width x color

### Getting a `Tensor` object's rank

To determine the rank of a `Tensor` object, call the `TensorFlow.Ops.rank` method. For example, the following method programmatically determines the rank of the `Tensor` defined in the previous section:

In [69]:
r = TensorFlow.Ops.rank(my_image); # After the graph runs, r will hold the value 4.

### Referring to `Tensor` slices

Since a `Tensor` is an n-dimensional array of cells, to access a single cell in a `Tensor` you need to specify n indices.

For a rank 0 tensor (a scalar), no indices are necessary, since it is already a single number.

For a rank 1 tensor (a vector), passing a single index allows you to access a number:

```julia
my_scalar = my_vector[2]
```

Note that the index passed inside the `[]` can itself be a scalar `Tensor`, if you want to dynamically choose an element from the vector.

For tensors of rank 2 or higher, the situation is more interesting. For a `Tensor` of rank 2, passing two numbers returns a scalar, as expected:

```julia
my_scalar = my_matrix[1, 2]
```

Passing a single number, however, returns a subvector of a matrix, as follows:

```julia
my_row_vector = my_matrix[2]
my_column_vector = my_matrix[:, 3]
```

The `:` notation is Julia slicing syntax for "leave this dimension alone". This is useful in higher-rank Tensors, as it allows you to access its subvectors, submatrices, and even other subtensors.

**★Important**: TensorFlow.jl uses the 1-based Julia indexing convention, which is different from the Python API.

## Shape

The **shape** of a tensor is the number of elements in each dimension. TensorFlow automatically infers shapes during graph construction. These inferred shapes might have known or unknown rank. If the rank is known, the sizes of each dimension might be known or unknown.

The TensorFlow documentation uses three notational conventions to describe tensor dimensionality: rank, shape, and dimension number. The following table shows how these relate to one another:

|Rank|Shape|Dimension number|Example|
|---|---|---|---|
|0|[]|0-D|A 0-D tensor. A scalar.|
|1|[D0]|1-D|A 1-D tensor with shape [5].|
|2|[D0, D1]|2-D|A 2-D tensor with shape [3,4].|
|3|[D0, D1, D2]|3-D|A 3-D tensor with shape [1,4,3].|
|n|[D0,D1,...,Dn-1]|n-D|A tensor with shape [D0,D1,...,Dn-1]|

Shapes are represented via Julia vectors of ints, or with the `TensorShape`.

### Getting a `Tensor` object's shape

There are two ways of accessing the shape of a `Tensor`. While building the graph, it is often useful to ask what is already known about a tensor's shape. This can be done using `get_shape` method on a `Tensor` object. This method returns a `TensorShape` object, which is a convenient way of representing partially-specified shapes (since, when building the graph, not all shapes will be fully known).

It is also possible to get a `Tensor` that will represent the fully-defined shape of another `Tensor` at runtime. This is done by calling the `TensorFlow.Ops.shape` operation. This way, you can build a graph that manipulates the shapes of tensors by building other tensors that depend on the dynamic shape of the input `Tensor`.

For example, here is how to make a vector of zeros with the same size as the number of columns in a given matrix:

```julia
my_zeros = Tensor(zeros(get_shape(my_matrix, 2)))
```

### Changing the shape of a `Tensor`

The **number of elements** of a tensor is the product of the sizes of all its shapes. The number of elements of a scalar is always `1`. Since there are often many different shapes that have the same number of elements, it's often convenient to be able to change the shape of a `Tensor`, keeping its elements fixed. This can be done with `reshape`.

The following examples demonstrate how to reshape tensors:

In [125]:
rank_three_tensor = Tensor(ones(3,4,5))
matrix = reshape(rank_three_tensor, [6, 10]) # Reshape existing content into a 6x10 matrix
matrixB = reshape(matrix, [3, -1]) # Reshape existing content into a 3x20 matrix. -1 tells
                                   # reshape to calculate the size of this dimension
matrixAlt = reshape(matrixB, [4, 3, -1]); # Reshape existing content into a 4x3x5 tensor

# Note that the number of elements of the reshaped Tensors has to match the
# original number of elements. Therefore, the following example generates an
# error because no possible value for the last dimension will match the number
# of elements.
# yet_another = reshape(matrixAlt, [13, 2, -1])  # ERROR!

## Data types

In addition to dimensionality, Tensors have a data type. Unlike the Python API, TensorFlow.jl uses native Julia data types like `Float32`.

It is not possible to have a `Tensor` with more than one data type. It is possible, however, to serialize arbitrary data structures as strings and store those in `Tensor`s.

It is possible to cast `Tensor`s from one datatype to another using `cast`:

In [133]:
# Cast a constant integer tensor into floating point.
float_tensor = cast(constant([1, 2, 3]), Float32)

<Tensor Cast_41:1 shape=(3) dtype=Float32>

To inspect a `Tensor`'s data type, use the `eltype` method.

## Evaluating Tensors

[WIP]

## Printing Tensors

For debugging purposes you might want to print the value of a `Tensor`. While `tfdbg` provides advanced debugging support, TensorFlow also has an operation to directly print the value of a `Tensor`.

Note that you rarely want to use the following pattern when printing a `Tensor`:

```julia
t = <<some tensorflow operation>>
println(t) # This will print the symbolic tensor when the graph is being built.
           # This tensor does not have a value in this context.
```

This code prints the `Tensor` object (which represents deferred computation) and not its value. Instead, TensorFlow provides the `TensorFlow.Ops.print` operation, which returns its first tensor argument unchanged while printing the set of `Tensor`s it is passed as the second argument.

To correctly use `TensorFlow.Ops.print` its return value must be used. See the example below

```julia
t = <<some tensorflow operation>>
TensorFlow.Ops.print(t, [t])  # This does nothing
t = TensorFlow.Ops.print(t, [t])  # Here we are using the value returned by tf.Print
result = t + 1  # Now when result is evaluated the value of `t` will be printed.
```

When you evaluate `result` you will evaluate everything `result` depends upon. Since `result` depends upon `t`, and evaluating `t` has the side effect of printing its input (the old value of `t`), `t` gets printed.

----------------------
*Except as otherwise noted, the content of this page is licensed under the [Creative Commons Attribution 3.0 License](https://creativecommons.org/licenses/by/3.0/), and code samples are licensed under the [Apache 2.0 License](https://www.apache.org/licenses/LICENSE-2.0). This notebook is adapted from the official [TensorFlow Guide on Low Level APIs](https://www.tensorflow.org/guide/low_level_intro).*